## AGAME Workflow

#### GPP maps (15 sites)

In [1]:
'''
This code includes all the steps needed to compute GPP maps with AGAME methodology:
The methodology includes the following steps:

1)Get icos tables (15 sites)
2)Get Sentinel-2 tables (15 sites)
3)Get ecosystem boundaries (15 sites)
4)Combine tables
5)Train XGBoost model
6)Evaluate XGBoost model
7)Create GPP maps (15 sites) <-----------------------------------------
8)Creata uncertainty maps
'''

'\nThis code includes all the steps needed to compute GPP maps with AGAME methodology:\nThe methodology includes the following steps:\n\n1)Get icos tables (15 sites)\n2)Get Sentinel-2 tables (15 sites)\n3)Get ecosystem boundaries (15 sites)\n4)Combine tables\n5)Train XGBoost model\n6)Evaluate XGBoost model\n7)Create GPP maps (15 sites) <-----------------------------------------\n8)Creata uncertainty maps\n'

In [2]:
import deims
import pandas as pd
import geopandas as gpd
import folium
from folium import plugins
import ee
import xarray
import geemap
import rioxarray
from pyproj import CRS
from xgboost import XGBRegressor
import os
import matplotlib.pyplot as plt
from shapely.geometry import mapping
from shapely.geometry import Polygon
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

In [3]:
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [4]:
def get_unique_years(date_range_values):
    unique_years = set()
    for date in date_range_values:
        year = date.split('-')[0] 
        unique_years.add(year)    

    unique_years = sorted(unique_years)

    return unique_years

def get_coordinates_area(df, site):
    site_url = df['deims'].loc[df['site name'] == site].values[0]
    latitude  = df['latitude'].loc[df['site name'] == site].values[0]
    longitude = df['longitude'].loc[df['site name'] == site].values[0]
    if pd.isna(site_url):
        raise ValueError("The site DEIMS url is empty. Please provide a valid site URL.")
    else:
        boundaries   = deims.getSiteBoundaries([site_url])
        information = deims.getSiteById(site_id=site_url)
    return boundaries, latitude, longitude, information

def map_coordinates_area(df, boundaries, site):
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df.longitude, df.latitude),
        crs="EPSG:3857"
    )
    gdf = gdf.loc[gdf['site name'] == site]
    centroid = gdf.geometry.centroid.union_all()
    m = folium.Map(location=[centroid.y, centroid.x], zoom_start=12)
    for idx, row in gdf.iterrows():
        folium.Marker(
            location=[row.geometry.y, row.geometry.x],
            popup=row['site name']
        ).add_to(m)

    folium.GeoJson(boundaries).add_to(m)

    return m

def get_gee_area(boundaries):
    total_bounds = boundaries.total_bounds
    aoi = ee.Geometry.Rectangle([total_bounds[0], total_bounds[1], total_bounds[2], total_bounds[3]])
    # Initialize the geemap Map
    Map = geemap.Map()

    # Add the rectangle to the map
    Map.addLayer(aoi, {}, 'Bounding Box')

    # Center the map around the bounding box
    Map.centerObject(aoi, 10)

    return Map, aoi

def get_gee_area_from_gpp(polygon_gdf):
    # Ensure the GeoDataFrame contains a valid geometry column
    if polygon_gdf.empty or polygon_gdf.geom_type.iloc[0] != 'Polygon':
        raise ValueError("GeoDataFrame is empty or does not contain Polygon geometries.")
    
    # Get the first polygon from the GeoDataFrame
    polygon = polygon_gdf.geometry.iloc[0]

    # Convert the Shapely Polygon to GeoJSON-like dict
    geojson = mapping(polygon)
    
    # Create an Earth Engine Geometry from the GeoJSON
    aoi = ee.Geometry.Polygon(geojson['coordinates'])

    # Initialize the geemap Map
    Map = geemap.Map()

    # Add the rectangle to the map
    Map.addLayer(aoi, {}, 'Bounding Box')

    # Center the map around the bounding box
    Map.centerObject(aoi, 15)
    
    return Map, aoi

def get_gee_area_from_gpp_multipolygon(geodf):
    
    if geodf.empty:
        raise ValueError("GeoDataFrame is empty.")
    
    # Get the first geometry from the GeoDataFrame
    geom = geodf.geometry.iloc[0]

    # Check the geometry type
    if geom.geom_type == 'Polygon':
        geojson = mapping(geom)
        aoi = ee.Geometry.Polygon(geojson['coordinates'])
    elif geom.geom_type == 'MultiPolygon':
        # Convert each Polygon in the MultiPolygon to GeoJSON and create ee.Geometry.MultiPolygon
        geojson = mapping(geom)
        polygons = [ee.Geometry.Polygon(polygon) for polygon in geojson['coordinates']]
        aoi = ee.Geometry.MultiPolygon(polygons)
        # aoi = ee.Geometry.Polygon(polygons[0])
    else:
        raise ValueError("GeoDataFrame contains unsupported geometry type. Only Polygon and MultiPolygon are supported.")

    # Initialize the geemap Map
    Map = geemap.Map()

    # Add the rectangle to the map
    Map.addLayer(aoi, {}, 'Bounding Box')

    # Center the map around the bounding box
    Map.centerObject(aoi, 15)
    
    return Map, aoi

def get_gee_area_from_gpp_multipolygon_single_polygon(geodf, num_polygon):
    
    if geodf.empty:
        raise ValueError("GeoDataFrame is empty.")
    
    # Get the first geometry from the GeoDataFrame
    geom = geodf.geometry.iloc[0]

    # Check the geometry type
    if geom.geom_type == 'Polygon':
        geojson = mapping(geom)
        aoi = ee.Geometry.Polygon(geojson['coordinates'])
    elif geom.geom_type == 'MultiPolygon':
        # Convert each Polygon in the MultiPolygon to GeoJSON and create ee.Geometry.MultiPolygon
        geojson = mapping(geom)
        first_polygon_coords = geojson['coordinates'][num_polygon]
        polygon = Polygon(first_polygon_coords[0]) 
        geojson_polygon = polygon.__geo_interface__
        aoi = ee.Geometry.Polygon(geojson_polygon['coordinates'])
    else:
        raise ValueError("GeoDataFrame contains unsupported geometry type. Only Polygon and MultiPolygon are supported.")
    
    area_m2 = aoi.area().getInfo()
    area_km2 = area_m2 / 1e6
    print(f"The area of the site is {area_m2} square meters")
    print(f"The area of the site is {area_km2} square kilometers")

    # Initialize the geemap Map
    Map = geemap.Map()

    # Add the rectangle to the map
    Map.addLayer(aoi, {}, "Site's area")

    if area_km2 < 1:
        print(f"The area of the site is below 1 square kilometer. A new area with a minimun of 1 square kilometer will be created")
        centroid = aoi.centroid()
        buffer = centroid.buffer(500)
        aoi = buffer.bounds()
        error_margin = 1  # meter
        area_m2 = aoi.area(maxError=error_margin).getInfo()
        area_km2 = area_m2 / 1e6
        print(f"The new area of the site is {area_m2} square meters")
        print(f"The mew area of the site is {area_km2} square kilometers")
        Map.addLayer(aoi, {}, "New site's area")
        
    # Center the map around the bounding box
    Map.centerObject(aoi, 15)

    print(f"The site geometry is ready to extract Earth Observation data\n")
    
    return Map, aoi

def apply_scale_factors_s2(image):
    optical_bands = image.select(['B.']).divide(10000)
    thermal_bands = image.select(['B.*']).divide(10000)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

def apply_scale_factors_e5(image):
    image = image.divide(24*60*60)
    return image

# function to derive VIs
def calculateVI(image):
    '''This method calculates different vegetation indices in a image collection and adds their values as new bands'''

    # defining dictionary of bands Sentinel-2 
    dict_bands = {

        "blue"  :  'B2',                              #Blue band                        
        "green" :  'B3',                              #Green band
        "red"   :  'B4',                              #Red band
        "red1"  :  'B5',                              #Red-edge spectral band   
        "red2"  :  'B6',                              #Red-edge spectral band
        "red3"  :  'B7',                              #Red-edge spectral band    
        "NIR"   :  'B8',                              #Near-infrared band
        "NIRn"  :  'B8A',                             #Near-infrared narrow
        "WV"    :  'B9',                              #Water vapour
        "SWIR1" :  'B11',                             #Short wave infrared 1
        "SWIR2" :  'B12',                             #Short wave infrared 2
    }

    # specify bands 
    dict  = dict_bands
    blue  = dict["blue"]                              #Blue band                        
    green = dict["green"]                             #Green band
    red   = dict["red"]                               #Red band
    red1  = dict["red1"]                              #Red-edge spectral band    
    red2  = dict["red2"]                              #Red-edge spectral band
    red3  = dict["red3"]                              #Red-edge spectral band
    NIR   = dict["NIR"]                               #Near-infrared band
    NIRn  = dict["NIRn"]                              #Near-infrared band
    WV    = dict["WV"]                                #Water vapour
    SWIR1 = dict["SWIR1"]                             #Short wave infrared 1
    SWIR2 = dict["SWIR2"]                             #Short wave infrared 2

    bands_for_expressions = {

        'blue'  : image.select(blue).divide(10000),
        'green' : image.select(green).divide(10000), 
        'red'   : image.select(red).divide(10000),
        'red1'  : image.select(red1).divide(10000), 
        'red2'  : image.select(red2).divide(10000),
        'red3'  : image.select(red3).divide(10000), 
        'NIR'   : image.select(NIR).divide(10000),
        'NIRn'  : image.select(NIRn).divide(10000),
        'WV'    : image.select(WV).divide(10000),
        'SWIR1' : image.select(SWIR1).divide(10000),
        'SWIR2' : image.select(SWIR2).divide(10000)}

    # greeness related indices
    # NDVI                                                                             (Rouse et al., 1974)
    NDVI  = image.normalizedDifference([NIR, red]).rename("NDVI") 
    # EVI                                                                             
    EVI   = image.expression('2.5*(( NIR - red ) / ( NIR + 6 * red - 7.5 * blue + 1 ))', 
            bands_for_expressions).rename("EVI")
    # EVI2                                                                             (Jiang et al., 2008)
    EVI2  = image.expression('2.5*(( NIR - red ) / ( NIR + 2.4 * red + 1 ))', 
            bands_for_expressions).rename("EVI2")

    # greeness related indices with Sentinel-2 narrow bands / Red-edge
    # Clr
    CLr  = image.expression('(red3/red1)-1', bands_for_expressions).rename("CLr")
    # Clg
    Clg  = image.expression('(red3/green)-1', bands_for_expressions).rename("CLg")
    # MTCI
    MTCI = image.expression('(red2-red1)/(red1-red)', bands_for_expressions).rename("MTCI")
    # MNDVI                                                                            (Add reference)
    MNDVI = image.normalizedDifference([red3, red1]).rename("MNDVI")    

    # water related indices
    # MNDWI                                                                            (Add reference)
    MNDWI = image.normalizedDifference([green, SWIR1]).rename("MNDWI")    
    # NDWI OR LSWI or NDII or NDMI                                                     (Add reference)
    LSWI  = image.normalizedDifference([NIR, SWIR1]).rename("LSWI")
    # NDII                                                                             (Hunt & Qu, 2013)
    NDII   = image.normalizedDifference([NIR, SWIR2]).rename("NDII")

    image = image.addBands(NDVI).addBands(EVI).addBands(EVI2)
    image = image.addBands(CLr).addBands(Clg).addBands(MTCI).addBands(MNDVI)
    image = image.addBands(MNDWI).addBands(LSWI).addBands(NDII)

    return image 

def get_s2_array(period,aoi,sentinel_bands,MGRS_TILE=None,cloud_percentage=100,resolution=100):
    ic = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate(period[0],period[1])
    # ic = ic.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',cloud_percentage))
    ic = ic.filterBounds(aoi) 
    if not MGRS_TILE   is None:
        print(f'Retriving collection for {MGRS_TILE} tile')
        ic = ic.filter(ee.Filter.eq('MGRS_TILE', MGRS_TILE)) 
    ic = ic.map(apply_scale_factors_s2)
    ic = ic.map(calculateVI)
    ic = ic.select(sentinel_bands)

    count = ic.size().getInfo()
    print('Number of images:', count)
    image_names = ic.aggregate_array('system:id').getInfo()
    print('Image names:', image_names)

    if count == 0:
        print("No images found in the period.")

    if count > 1:
        print("More than one image in the period.")


        print('Selecting the first image in the collection:', image_names[0])
        
        ic = ic.filter(ee.Filter.eq('system:id', image_names[0]))
        count = ic.size().getInfo()
        print('Number of images:', count)
        image_names = ic.aggregate_array('system:id').getInfo()
        print('Image names:', image_names)

    # ic = ee.ImageCollection(ic.mean())
    ic_sample = ic.getRegion(aoi, resolution).getInfo()
    return ic_sample, ic

def get_s2_df(s2_array):
    df = pd.DataFrame(s2_array[1:], columns=s2_array[0])
    df = df.iloc[:,1:]
    df['time'] = pd.to_datetime(df['time'], unit='ms').dt.date
    df['time'] = pd.to_datetime(df['time'])
    df.set_index(['time','latitude','longitude'], inplace=True)
    return df

def get_e5_array(period,aoi,e5_bands,resolution=100):
    e5 = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').filterDate(period[0],period[1])
    e5 = e5.filterBounds(aoi).select(e5_bands)
    e5 = e5.map(apply_scale_factors_e5)
    e5_sample = e5.getRegion(aoi, resolution).getInfo()
    return e5_sample, e5

def get_e5_df(e5_array):
    dfe5 = pd.DataFrame(e5_array[1:], columns=e5_array[0])
    dfe5['time'] = pd.to_datetime(dfe5['id'], format='%Y%m%d')
    dfe5 = dfe5.iloc[:,1:]
    dfe5.set_index(['time','latitude','longitude'], inplace=True)
    dfe5.rename(columns={'surface_net_solar_radiation_sum':'SW_IN_ERA_GEE'}, inplace=True)
    return dfe5

def merge_s2_e5(s2_df, e5_df):
    df1 = s2_df.reset_index()
    df2 = e5_df.reset_index()
    df_merged = df1.merge(df2, on=['time', 'latitude', 'longitude'], how='left')

    df_merged = df_merged.dropna()
    df_merged.set_index(['time','latitude','longitude'], inplace=True)
    return df_merged

def map_image(ic, aoi, band, label):
    # Define visualization parameters
    vis_params = {
        'min': 0.5,
        'max': 1.0,
        'palette': ['red', 'white', 'green']
    }

    # Create a map
    Map = geemap.Map()

    # Center the map on the area of interest
    Map.centerObject(aoi, 12)

    # Add the mean NDVI layer to the map
    Map.addLayer(ic.select(band), vis_params, label)

    # Display the map
    return Map

def get_environmental_data(directory_data, filename,sentinel_vi,sentinel_bands, general):
    env_df = pd.read_csv(os.path.join(directory_data, filename), index_col='TIMESTAMP', parse_dates=['TIMESTAMP'])

    s2_all = env_df.columns.values.tolist()
    s2_all = sorted([item for item in s2_all if not (item.endswith('_residual') or 
                                                    item.endswith('_trend') or 
                                                    item.endswith('_season'))])

    s2_all = sorted([item for item in s2_all if not (item.startswith('CO2') or 
                                                    item.startswith('H_') or 
                                                    item.startswith('LE_'))])

    s2_all = [item for item in s2_all if item not in sentinel_vi]
    s2_all = [item for item in s2_all if item not in sentinel_bands]
    s2_all = [item for item in s2_all if item not in general]

    selected_columns = s2_all 
    env_df = env_df[selected_columns]

    return env_df

def merge_s2_env_data(s2_df, env_df, expected_columns):
    s2_df = s2_df.reset_index()
    env_df = env_df.reset_index()
    env_df = env_df.rename(columns={'TIMESTAMP':'time'})

    df_merged = s2_df.merge(env_df, on=['time'], how='left')
    df_merged.set_index(['time','latitude','longitude'], inplace=True)

    df_merged = df_merged[expected_columns]
    
    return df_merged

def predict_gpp(df_merged, model_file):
    print('Predicting Gross Primary Production')
    loaded_model = XGBRegressor()
    loaded_model.load_model(model_file)
    y_pred = loaded_model.predict(df_merged)
    df_merged['GPP'] = y_pred
    ds_gpp = df_merged.to_xarray()
    return ds_gpp

def plot_save_var(ds, var, netcdf_output, geotif_output):
    ds[var].isel(time=0).plot()
    crs = CRS.from_epsg(4326) #3857

    ds.attrs['crs'] = crs.to_string()
    ds[var].to_netcdf(netcdf_output)

    da = ds[var].isel(time=0)
    da.rio.write_crs(crs.to_string(), inplace=True)
    da.rio.to_raster(geotif_output)

def sites_info(site_list, directory_data):
    df_sites = pd.read_excel(site_list)
    df_sites = df_sites.dropna(subset=['deims'])
    site_name_list = sorted(df_sites['site name'].values.tolist())
    files_list = os.listdir(directory_data)
    files_list = sorted([file for file in files_list if any(file.startswith(site_name) for site_name in site_name_list)])

    return site_name_list, files_list, df_sites

def plot_save_gpp(ds, site, period, directory_maps): 
    print('Saving Gross Primary Production products\n')   
    crs = CRS.from_epsg(4326)
    ds.attrs['crs'] = crs.to_string()

    ds['GPP'].to_netcdf(os.path.join(directory_maps, f'{site} - Gross Primary Production ({period[0]}).nc'))

    da = ds['GPP'].isel(time=0)

    plt.figure(figsize=(10, 6))
    da.plot()
    plt.title(f'{site} - Gross Primary Production (GPP) - {period[0]}')
    plt.savefig(os.path.join(directory_maps, f'{site} - Gross Primary Production ({period[0]}).png'))
    plt.close()

    da.rio.write_crs(crs.to_string(), inplace=True)

    da.rio.to_raster(
        os.path.join(directory_maps, f'{site} - Gross Primary Production ({period[0]}).tif'),
        driver='COG',
        compress='deflate',
        nodata=da.attrs.get('_FillValue', None),
        dtype=da.dtype.name
    )

def get_collection_without_clouds(
        collection,
        year_list,
        aoi, 
        longitude,
        latitude,
        max_cloud_coverage,
        local_cloud_coverage,
):
    
    # function to load data set with specified period and location
    def load_catalog(catalog, time, location, bands):
        dataset = ee.ImageCollection(catalog).filterDate(time[0],time[1]).filterBounds(location).select(bands)
        return dataset

    # function to derive VIs
    def calculateVI(image):
        '''This method calculates different vegetation indices in a image collection and adds their values as new bands'''

        # defining dictionary of bands Sentinel-2 
        dict_bands = {

            "blue"  :  'B2',                              #Blue band                        
            "green" :  'B3',                              #Green band
            "red"   :  'B4',                              #Red band
            "red1"  :  'B5',                              #Red-edge spectral band   
            "red2"  :  'B6',                              #Red-edge spectral band
            "red3"  :  'B7',                              #Red-edge spectral band    
            "NIR"   :  'B8',                              #Near-infrared band
            "NIRn"  :  'B8A',                             #Near-infrared narrow
            "WV"    :  'B9',                              #Water vapour
            "SWIR1" :  'B11',                             #Short wave infrared 1
            "SWIR2" :  'B12',                             #Short wave infrared 2
        }

        # specify bands 
        dict  = dict_bands
        blue  = dict["blue"]                              #Blue band                        
        green = dict["green"]                             #Green band
        red   = dict["red"]                               #Red band
        red1  = dict["red1"]                              #Red-edge spectral band    
        red2  = dict["red2"]                              #Red-edge spectral band
        red3  = dict["red3"]                              #Red-edge spectral band
        NIR   = dict["NIR"]                               #Near-infrared band
        NIRn  = dict["NIRn"]                              #Near-infrared band
        WV    = dict["WV"]                                #Water vapour
        SWIR1 = dict["SWIR1"]                             #Short wave infrared 1
        SWIR2 = dict["SWIR2"]                             #Short wave infrared 2

        bands_for_expressions = {

            'blue'  : image.select(blue).divide(10000),
            'green' : image.select(green).divide(10000), 
            'red'   : image.select(red).divide(10000),
            'red1'  : image.select(red1).divide(10000), 
            'red2'  : image.select(red2).divide(10000),
            'red3'  : image.select(red3).divide(10000), 
            'NIR'   : image.select(NIR).divide(10000),
            'NIRn'  : image.select(NIRn).divide(10000),
            'WV'    : image.select(WV).divide(10000),
            'SWIR1' : image.select(SWIR1).divide(10000),
            'SWIR2' : image.select(SWIR2).divide(10000)}

        # greeness related indices
        # NDVI                                                                             (Rouse et al., 1974)
        NDVI  = image.normalizedDifference([NIR, red]).rename("NDVI") 
        # EVI                                                                             
        EVI   = image.expression('2.5*(( NIR - red ) / ( NIR + 6 * red - 7.5 * blue + 1 ))', 
                bands_for_expressions).rename("EVI")
        # EVI2                                                                             (Jiang et al., 2008)
        EVI2  = image.expression('2.5*(( NIR - red ) / ( NIR + 2.4 * red + 1 ))', 
                bands_for_expressions).rename("EVI2")

        # greeness related indices with Sentinel-2 narrow bands / Red-edge
        # Clr
        CLr  = image.expression('(red3/red1)-1', bands_for_expressions).rename("CLr")
        # Clg
        Clg  = image.expression('(red3/green)-1', bands_for_expressions).rename("CLg")
        # MTCI
        MTCI = image.expression('(red2-red1)/(red1-red)', bands_for_expressions).rename("MTCI")
        # MNDVI                                                                            (Add reference)
        MNDVI = image.normalizedDifference([red3, red1]).rename("MNDVI")    

        # water related indices
        # MNDWI                                                                            (Add reference)
        MNDWI = image.normalizedDifference([green, SWIR1]).rename("MNDWI")    
        # NDWI OR LSWI or NDII or NDMI                                                     (Add reference)
        LSWI  = image.normalizedDifference([NIR, SWIR1]).rename("LSWI")
        # NDII                                                                             (Hunt & Qu, 2013)
        NDII   = image.normalizedDifference([NIR, SWIR2]).rename("NDII")

        image = image.addBands(NDVI).addBands(EVI).addBands(EVI2)
        image = image.addBands(CLr).addBands(Clg).addBands(MTCI).addBands(MNDVI)
        image = image.addBands(MNDWI).addBands(LSWI).addBands(NDII)

        return image  

    # cloud coverage filter function
    def cloud_filter(collection, cloud_coverage_metadata_name, threshold):
        collection_cf = collection.filterMetadata(cloud_coverage_metadata_name,'less_than', threshold)
        # Show messages
        print('The maximun cloud coverage in the image is:', max_cloud_coverage)
        print('The original size of the collection is', collection.size().getInfo())
        # print(s2.first().getInfo())
        print('The filtered size of the collection is', collection_cf.size().getInfo(),'\n')
        return collection_cf
    
    def local_cloud_filter(s2, aoi, LOCAL_CLOUD_THRESH):
        # Describe functions
        # Function to scale the reflectance bands
        def apply_scale_factors_s2(image):
            optical_bands = image.select(['B.']).divide(10000)
            thermal_bands = image.select(['B.*']).divide(10000)
            return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

        # Function to create mask with cirrus clouds and cirrus pixels
        def extract_bit_s2_10_11(image):
            bit_position_clouds = 10
            bit_position_cirrus = 11

            # Bits 10 and 11 are clouds and cirrus, respectively.
            cloud_bit_mask = 1 << bit_position_clouds
            cirrus_bit_mask = 1 << bit_position_cirrus

            mask_clouds = image.bitwiseAnd(cloud_bit_mask).rightShift(bit_position_clouds)
            mask_cirrus = image.bitwiseAnd(cirrus_bit_mask).rightShift(bit_position_cirrus)
            mask = mask_clouds.add(mask_cirrus)
            return mask

        # Function to mask pixels with high reflectance in the blue (B2) band. The function creates a QA band
        def b2_mask(image):
            B2Threshold = 0.2
            B2Mask = image.select('B2').gt(B2Threshold)
            return image.addBands(B2Mask.rename('B2Mask'))

        # Function to create a band with ones
        def make_ones(image):
            # Create a band with ones
            ones_band = image.select('B2').divide(image.select('B2'))
            return image.addBands(ones_band.rename('Ones'))

        # Function to calculate area
        def get_area(img):
            cloud_area = make_ones(img).select('Ones').multiply(ee.Image.pixelArea()) \
                .reduceRegion(reducer=ee.Reducer.sum(), geometry=aoi, scale=30).values().get(0)
            return img.set('area_image', ee.Number(cloud_area))

        # Function to get local cloud percentage with QA band
        def get_local_cloud_percentage(img):
            error_margin = 1 # meter
            cloud_area = extract_bit_s2_10_11(img.select('QA60')).multiply(ee.Image.pixelArea()) \
                .reduceRegion(reducer=ee.Reducer.sum(), geometry=aoi, scale=60).values().get(0)
            return img.set('local_cloud_percentage', ee.Number(cloud_area).divide(aoi.area(maxError=error_margin)).multiply(100).round())

        # Function to get local cloud percentage with QA and area of image band
        def get_local_cloud_percentage_area_image(img):
            area_image = img.get('area_image')
            cloud_area = extract_bit_s2_10_11(img.select('QA60')).multiply(ee.Image.pixelArea()) \
                .reduceRegion(reducer=ee.Reducer.sum(), geometry=aoi, scale=60).values().get(0)
            return img.set('local_cloud_percentage_ai', ee.Number(cloud_area).divide(ee.Number(area_image)).multiply(100).round())

        # Function to get local cloud percentage with B2 and area of image band
        def get_local_cloud_percentage_area_image_b2(img):
            area_image = img.get('area_image')
            cloud_area = b2_mask(img).select('B2Mask').multiply(ee.Image.pixelArea()) \
                .reduceRegion(reducer=ee.Reducer.sum(), geometry=aoi, scale=60).values().get(0)
            return img.set('local_cloud_percentage_ai_b2', ee.Number(cloud_area).divide(ee.Number(area_image)).multiply(100).round())

        # def add_ndvi(image):
        #     # Calculate NDVI
        #     ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
        #     return image.addBands(ndvi)

        s2 = s2.filterBounds(aoi).map(lambda image: image.clip(aoi)).map(apply_scale_factors_s2) #.map(add_ndvi)
        
        # Processing
        # Calculate area
        s2 = s2.map(get_area)
        # Calculate local cloud percentage with QA band
        s2 = s2.map(get_local_cloud_percentage)
        # Calculate local cloud percentage with QA band and area image band
        s2 = s2.map(get_local_cloud_percentage_area_image)
        # Calculate local cloud percentage with B2 band and area image band
        s2 = s2.map(get_local_cloud_percentage_area_image_b2)
        # Filter images
        s2_filtered = s2.filter(ee.Filter.lte('local_cloud_percentage_ai', LOCAL_CLOUD_THRESH))
        s2_filtered = s2_filtered.filter(ee.Filter.lte('local_cloud_percentage_ai_b2', LOCAL_CLOUD_THRESH))

        # Show messages
        print('The maximun cloud coverage in the area is:', LOCAL_CLOUD_THRESH)
        print('The original size of the collection is', s2.size().getInfo())
        # print(s2.first().getInfo())
        print('The filtered size of the collection is', s2_filtered.size().getInfo(),'\n')
        
        return s2_filtered 

    # function for masking non-vegetation areas
    def maskS2nonvegetation(image):

            qa    = image.select('QA60')
            scl   = image.select('SCL')
            ndvi  = image.select('NDVI')
            mndvi = image.select('MNDVI')

            cloudBitMask = 1 << 10
            cirrusBitMask = 1 << 11

            #vegetationMask1 = 4 # vegetation
            #vegetationMask2 = 5 # non-vegetated
            #vegetationMask3 = 6 # water
            #vegetationMask4 = 7 # unclassified
            #vegetationMask5 = 11 # snow

            # this mask selects vegetation + non-vegetated + water + unclassified + areas with VIs (NDVI and MNDVI) greater that a threshold set in the configuration file
            mask = scl.eq(4).Or(scl.eq(5)).Or(scl.eq(6)).Or(scl.eq(7)).Or(scl.eq(11)).And(qa.bitwiseAnd(cloudBitMask).eq(0)).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
            # mask = scl.eq(4).And(qa.bitwiseAnd(cloudBitMask).eq(0)).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
            # mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

            vegetation = image.updateMask(mask)

            return vegetation
    
    # get inpu data
    # create a only file per year identified in the input files
    years = year_list

    # create range according to data in the input datafiles   
    start   = '%s-01-01'   %(years[0])                                                                                          
    end     = '%s-12-31'   %(years[-1])                                            
    timeSD  = [start, end]

    # create coordinates of the eddy covariance tower
    lon_lat =  [longitude, latitude]         
    point   = ee.Geometry.Point(lon_lat)

    # collections google earth engine    
    COPERNICUS_S2_L2A = collection #Multi-spectral surface reflectances (https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR)       
    COPERNICUS_S2_bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'QA10', 'QA20', 'QA60']

    # applying functions 
    # request of catalogues 
    S2_VI     = load_catalog(COPERNICUS_S2_L2A, timeSD, point, COPERNICUS_S2_bands)

    # filter cloud coverage
    cloud_coverage_metadata_name = 'CLOUDY_PIXEL_PERCENTAGE'                     # name of metadata property indicating cloud coverage in %

    # applying cloud filter 
    S2_VI = cloud_filter(S2_VI, cloud_coverage_metadata_name, max_cloud_coverage)   # max cloud coverage defined in the Config file

    # apply cloud local filter
    S2_VI = local_cloud_filter(S2_VI, aoi, local_cloud_coverage)

    # calculation of vegetation indices for the collection
    S2_VI = S2_VI.map(calculateVI)

    # applying mask 
    S2_VI = S2_VI.map(maskS2nonvegetation)

    return S2_VI, point

def get_ecosystem_geometry(training_dataset,number_clusters,training_scale,scale_getRegion, vector_scale, point, ic, bands, ecosystem_extent):
    #https://code.earthengine.google.com/2b95fd6462c6c906d4ed9a74fae51bf4
    Region    =  point.buffer(ecosystem_extent/2)
    inputML   =  ic.select(bands).median().clip(Region)

    # This trainning function takes pixes or pixels even in larger region than inputML
    training  = inputML.sample(region=Region, scale=training_scale, numPixels=training_dataset)
    clusterer = ee.Clusterer.wekaKMeans(number_clusters).train(training)

    result    = inputML.cluster(clusterer)
    results_colect  = ee.ImageCollection([result])
    df_clus = results_colect.getRegion(point, scale_getRegion).getInfo()
    df_clus = pd.DataFrame(df_clus)
    headers = df_clus.iloc[0]
    df_clus = pd.DataFrame(df_clus.values[1:], columns=headers).set_index('id')
    cluster_ecosystem = df_clus['cluster'][0]
    results_shp = result.reduceToVectors(scale=vector_scale, bestEffort=True)

    def classification(weka, num):
        class_vegetation = weka.select('label').filter(ee.Filter.eq('label', num))
        return class_vegetation

    cluster_name = []
    for i in range(number_clusters):
        globals()['cluster_%s'%i] = classification(results_shp, i).union(1).geometry()
        cluster_name.append(globals()['cluster_%s'%i])

    cluster_ecosystem_geometry  = cluster_name[cluster_ecosystem]

    et_image = ee.ImageCollection([result.eq(cluster_ecosystem)])

    return inputML, et_image, cluster_ecosystem_geometry 

def get_ecosystem_map(latitude,longitude,cluster_ecosystem_geometry, inputML, directory_maps, site, year):
    # define a method for displaying Earth Engine image tiles on a folium map.
    def add_ee_layer(self, ee_object, vis_params, name):
        try:    

            # display ee.Image()
            if isinstance(ee_object, ee.image.Image):    
                map_id_dict = ee.Image(ee_object).getMapId(vis_params)
                folium.raster_layers.TileLayer(
                tiles = map_id_dict['tile_fetcher'].url_format,
                attr = 'Google Earth Engine',
                name = name,
                overlay = True,
                control = True
                ).add_to(self)

            # display ee.ImageCollection()
            elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
                ee_object_new = ee_object.mosaic()
                map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
                folium.raster_layers.TileLayer(
                tiles = map_id_dict['tile_fetcher'].url_format,
                attr = 'Google Earth Engine',
                name = name,
                overlay = True,
                control = True
                ).add_to(self)

            # display ee.Geometry()
            elif isinstance(ee_object, ee.geometry.Geometry):    
                folium.GeoJson(
                data = ee_object.getInfo(),
                name = name,
                overlay = True,
                control = True,
                style_function=lambda x:vis_params
            ).add_to(self)

            # display ee.FeatureCollection()
            elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
                ee_object_new = ee.Image().paint(ee_object, 0, 2)
                map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
                folium.raster_layers.TileLayer(
                tiles = map_id_dict['tile_fetcher'].url_format,
                attr = 'Google Earth Engine',
                name = name,
                overlay = True,
                control = True
            ).add_to(self)

        except:
            print("Could not display {}".format(name))

    # add EE drawing method to folium.
    folium.Map.add_ee_layer = add_ee_layer

    # Add custom basemaps to folium
    basemaps = {
        'Google Maps': folium.TileLayer(
            tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
            attr = 'Google',
            name = 'Google Maps',
            overlay = True,
            control = True
        ),
        'Google Satellite': folium.TileLayer(
            tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
            attr = 'Google',
            name = 'Google Satellite',
            overlay = True,
            control = True
        ),
        'Google Terrain': folium.TileLayer(
            tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
            attr = 'Google',
            name = 'Google Terrain',
            overlay = True,
            control = True
        ),
        'Google Satellite Hybrid': folium.TileLayer(
            tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
            attr = 'Google',
            name = 'Google Satellite',
            overlay = True,
            control = True
        ),
        'Esri Satellite': folium.TileLayer(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            overlay = True,
            control = True
        )
    }


    # Mapping with folium
    # a) create a folium map object.
    my_map = folium.Map(location= [latitude,longitude], zoom_start=12)
    # b) add custom basemaps
    basemaps['Esri Satellite'].add_to(my_map)
    basemaps['Google Satellite Hybrid'].add_to(my_map)
    # c) set visualization parameters.
    vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}
    # d) display Geometry
    vis_params_geometry = dict(color='red', weight=2, opacity=10, fillColor='red')
    my_map.add_ee_layer(cluster_ecosystem_geometry,  vis_params_geometry , 'Ecosystem area')
    # d) display ee.Image
    dataset        = inputML.select('NDVI')
    vis_params = {
        'min': 0.5,
        'max': 1.0,
        'palette': ['red', 'white', 'green']
    }
    my_map.add_ee_layer(dataset, vis_params, 'NDVI')
    # e) add a layer control panel to the map.
    my_map.add_child(folium.LayerControl())
    plugins.Fullscreen().add_to(my_map)
    
    my_map.save(os.path.join(directory_maps, f'{site} - Ecosystem map ({year}).html'))
    return my_map

def get_et_df(et_array, year):
    df = pd.DataFrame(et_array[1:], columns=et_array[0])
    df = df.iloc[:,1:]
    df['time'] =  pd.to_datetime(f'{year}')
    df.set_index(['time','latitude','longitude'], inplace=True)
    return df

def plot_save_et(image, aoi, site, year, directory_maps, mse, test_r2, mae, rmse): 
    print(f'Saving Uncertainty product for {year}\n')   

    et_sample = image.getRegion(aoi, 10).getInfo()
    df_et =  get_et_df(et_sample, year)
    ds = df_et.to_xarray()

    crs = CRS.from_epsg(4326)
    ds.attrs['crs'] = crs.to_string()
    ds.attrs['title'] = f'Gross Primary Production Uncertainty Map - {year}'
    ds.attrs['description'] = 'This COG file represents the uncertainty in Gross Primary Production estimates for the specified year.'
    ds.attrs['source'] = site
    ds.attrs['year'] = year
    ds.attrs['MSE'] = mse
    ds.attrs['MAE'] = mae
    ds.attrs['RMSE'] = rmse
    ds.attrs['R^2'] = test_r2

    ds['cluster'].to_netcdf(os.path.join(directory_maps, f'{site} - Gross Primary Production Uncertainty Map ({year}).nc'))

    da = ds['cluster'].isel(time=0)
    da.attrs['crs'] = crs.to_string()
    da.attrs['title'] = f'Gross Primary Production Uncertainty Map - {year}'
    da.attrs['description'] = 'This COG file represents the uncertainty in Gross Primary Production estimates for the specified year.'
    da.attrs['source'] = site
    da.attrs['year'] = year
    da.attrs['MSE'] = mse
    da.attrs['MAE'] = mae
    da.attrs['RMSE'] = rmse
    da.attrs['R^2'] = test_r2

    plt.figure(figsize=(10, 6))
    da.plot()
    plt.title(f'{site} - Gross Primary Production (GPP) - Uncertainty Map - {year}')
    plt.savefig(os.path.join(directory_maps, f'{site} - Gross Primary Production Uncertainty Map ({year}).png'))
    plt.close()

    da.rio.write_crs(crs.to_string(), inplace=True)

    da.rio.to_raster(
        os.path.join(directory_maps, f'{site} - Gross Primary Production Uncertainty Map ({year}).tif'),
        driver='COG',
        compress='deflate',
        nodata=da.attrs.get('_FillValue', None),
        dtype=da.dtype.name
    )

def get_testing_data(directory_data, filename, expected_columns, gpp_column):
    env_df = pd.read_csv(os.path.join(directory_data, filename), index_col='TIMESTAMP', parse_dates=['TIMESTAMP'])
    env_testing = env_df[expected_columns]
    gpp_testing = env_df[gpp_column]
    return env_testing, gpp_testing

def predict_gpp_df(df, model_file):
    print('Testing Gross Primary Production')
    loaded_model = XGBRegressor()
    loaded_model.load_model(model_file)
    y_pred = loaded_model.predict(df)
    df['GPP_predicted'] = y_pred
    return df

def get_performance(df, year):
    df = df.reset_index()
    df = df[df['TIMESTAMP'].dt.year == int(year)]
    y_test = df['GPP_testing']
    y_pred = df['GPP_predicted']

    mse = mean_squared_error(y_test, y_pred)
    test_r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    print('Mean Squared Error:', mse)
    print('Root Mean Squared Error:', rmse)
    print('MAE:', mae)
    print("Test R^2 Score:", test_r2,'\n')

    return df, mse, test_r2, mae, rmse


In [5]:
site_list = r'D:\Proyectos2024\Agame\Repository\vlabs\sen2gpp\Input\list_sites_icos.xlsx'
model_file = r"D:\Proyectos2024\Agame\Repository\vlabs\sen2gpp\Output\Model\xgboost_model_all_nogeneral.json"
directory_data = r'D:\Proyectos2024\Agame\Repository\vlabs\sen2gpp\Output\Tables'
directory_maps = r'D:\Proyectos2024\Agame\Repository\vlabs\sen2gpp\Output\Maps'

sentinel_vi = ['CLr','EVI','EVI2','LSWI', 'MNDVI','MNDWI','NDII','NDVI']
sentinel_bands  = ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12']
general = ['latitude', 'longitude', 'elevation', 'canopy_height', 'biom_croplands', 'biom_dbf', 'biom_enf', 'biom_grasslands', 'winter','spring', 'summer', 'fall']
expected_columns = ['CLr', 'DAY_D', 'DAY_RANDUNC_N', 'EBC_CF_N', 'EVI', 'EVI2', 'G_F_MDS', 'LSWI', 'LW_IN_ERA', 'LW_IN_F', 'LW_IN_F_MDS', 'LW_IN_JSB', 'LW_IN_JSB_ERA', 'LW_IN_JSB_F', 'LW_OUT', 'MNDVI', 'MNDWI', 'NDII', 'NDVI', 'NETRAD', 'NIGHT_D', 'NIGHT_RANDUNC_N', 'PA_ERA', 'PA_F', 'PPFD_IN', 'PPFD_OUT', 'P_ERA', 'P_F', 'SWC_F_MDS_1', 'SWC_F_MDS_2', 'SW_IN_ERA', 'SW_IN_F', 'SW_IN_F_MDS', 'SW_IN_POT', 'SW_OUT', 'TA_ERA', 'TA_ERA_DAY', 'TA_ERA_DAY_SD', 'TA_ERA_NIGHT', 'TA_ERA_NIGHT_SD', 'TA_F', 'TA_F_DAY', 'TA_F_DAY_SD', 'TA_F_MDS', 'TA_F_MDS_DAY', 'TA_F_MDS_DAY_SD', 'TA_F_MDS_NIGHT', 'TA_F_MDS_NIGHT_SD', 'TA_F_NIGHT', 'TA_F_NIGHT_SD', 'TS_F_MDS_1', 'TS_F_MDS_2', 'TS_F_MDS_3', 'USTAR', 'VPD_ERA', 'VPD_F', 'VPD_F_MDS', 'WS_ERA', 'WS_F', 'day', 'month']
# date_range_values = ['2022-11-07','2023-12-31']

date_range_values = ['2021-10-26','2021-12-31'] 

# MGRS_TILE = '34VFP' #'35VLJ'
MGRS_TILE = None

max_cloud_coverage   = 100
local_cloud_coverage = 0
training_dataset = 10000
number_clusters = 2 # Torgnon = 3
training_scale = 10
scale_getRegion = 10
vector_scale = 10
ecosystem_extent = 10000

In [6]:
site_name_list, files_list, df_sites = sites_info(site_list, directory_data)
site_name_list = site_name_list[2:3]
files_list = files_list[2:3]

In [9]:
df_sites

,country code,site name,station id,main ecosystem,latitude,longitude,elevation,canopy_height,instrument_height,fetch_desired,status,snow_season,role,deims,Unnamed: 14
1,DE,Hohes Holz,DE-HoH,dbf,52.08656,11.222350,193,33.0,45.00,500,Snow coverage for some years (e.g. 2020),dec-jan,training,https://deims.org/ddd2e8d2-44db-420e-8fa4-6b4f...,https://meta.icos-cp.eu/resources/stations/ES_...
2,IT,Torgnon,IT-Tor,grasslands,45.84444,7.578055,2168,0.0,2.55,80,High snow coverage,nov-may,training,https://deims.org/a03ef869-aa6f-49cf-8e86-f791...,https://meta.icos-cp.eu/resources/stations/ES_...
3,FI,Hyytiala,FI-Hyy,enf,61.84741,24.294770,181,18.0,27.00,400,High snow coverage,jan-apr,training,https://deims.org/663dac80-211d-4c19-a356-04ee...,https://meta.icos-cp.eu/resources/stations/ES_...
6,SE,Svartberget,SE-Svb,enf,64.25611,19.774500,267,28.8,34.50,700,Maximun canopy height of tree,NaN,testing,https://deims.org/c0705d0f-92c1-4964-a345-38c0...,https://meta.icos-cp.eu/resources/stations/ES_...


In [6]:


for index in range(len(site_name_list)):

    print(f'Calculating products for {site_name_list[index]}\n')

    site = site_name_list[index]
    filename = files_list[index]
    
    boundaries, latitude, longitude, information = get_coordinates_area(df_sites, site)
    m = map_coordinates_area(df_sites, boundaries, site)
    Map, aoi = get_gee_area_from_gpp_multipolygon_single_polygon(boundaries, 6)

    date_range = pd.date_range(start=date_range_values[0], end=date_range_values[1], freq='D')
    dates_list = date_range.strftime('%Y-%m-%d').tolist()
    year_list = get_unique_years(dates_list)

    env_testing, gpp_testing = get_testing_data(directory_data, filename, expected_columns, 'GPP_DT_VUT_USTAR50')
    gpp_predicted = predict_gpp_df(env_testing, model_file)
    gpp_predicted['GPP_testing'] = gpp_testing 

    for index in range(len(dates_list)-1):
        period = [dates_list[index], dates_list[index+1]]
        print(f"Period: {dates_list[index]}, {dates_list[index+1]}")
        
        try:
            s2_array, s2 = get_s2_array(period,aoi,sentinel_vi,MGRS_TILE=MGRS_TILE,cloud_percentage=100,resolution=10)
            s2_df = get_s2_df(s2_array)
            env_df = get_environmental_data(directory_data,filename,sentinel_vi,sentinel_bands, general)
            df_merged = merge_s2_env_data(s2_df, env_df, expected_columns)
            ds_gpp = predict_gpp(df_merged, model_file)
            plot_save_gpp(ds_gpp, site, period, directory_maps)

        except ee.EEException as ee_error:
            print(f"Earth Engine Exception: {ee_error}\n")

    for year in year_list:
        print(f'Calculating uncerstainty product for {site} in {year}\n')

        df, mse, test_r2, mae, rmse = get_performance(gpp_predicted, year)

        ic, point = get_collection_without_clouds('COPERNICUS/S2_SR_HARMONIZED',[year],aoi,longitude,latitude,max_cloud_coverage,local_cloud_coverage)
        bands = ic.first().bandNames().getInfo() 
        inputML, cluster_ecosystem_image, cluster_ecosystem_geometry = get_ecosystem_geometry(training_dataset,number_clusters,training_scale,scale_getRegion, vector_scale, point, ic, bands, ecosystem_extent)
        ecosystem_map = get_ecosystem_map(latitude,longitude,cluster_ecosystem_geometry, inputML, directory_maps, site, year)
        # Map = map_image(cluster_ecosystem_image.first(), aoi, "cluster", "Cluster")
        plot_save_et(cluster_ecosystem_image,aoi, site, year, directory_maps, mse, test_r2, mae, rmse)

Calculating products for Svartberget

The area of the site is 67340866.87546413 square meters
The area of the site is 67.34086687546413 square kilometers
The site geometry is ready to extract Earth Observation data

Testing Gross Primary Production
Period: 2021-10-26, 2021-10-27
Number of images: 1
Image names: ['COPERNICUS/S2_SR_HARMONIZED/20211026T103131_20211026T103127_T34WDS']
Earth Engine Exception: ImageCollection.getRegion: Too many values: 1553365 points x 8 bands x 1 images > 1048576.

Period: 2021-10-27, 2021-10-28
Number of images: 0
Image names: []
No images found in the period.
Earth Engine Exception: ImageCollection.getRegion: No bands in collection.

Period: 2021-10-28, 2021-10-29
Number of images: 1
Image names: ['COPERNICUS/S2_SR_HARMONIZED/20211028T102039_20211028T102038_T34WDS']
Earth Engine Exception: ImageCollection.getRegion: Too many values: 1553365 points x 8 bands x 1 images > 1048576.

Period: 2021-10-29, 2021-10-30
Number of images: 0
Image names: []
No image

EEException: User memory limit exceeded.

In [ ]:
import os
import re
from datetime import datetime

# Define the folder containing the files
folder_path = r'D:\Proyectos2024\Agame\Repository\vlabs\sen2gpp\Output\Maps\Torgnon - Gross Primary Production (2021 - 2023)'

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.tif'):
        # Extract the initial part and the date
        match = re.match(r'Torgnon - Gross Primary Production \((\d{4})-(\d{2})-(\d{2})\)\.tif', filename)
        if match:
            # Extract the date components
            year, month, day = match.groups()
            # Create the new filename
            new_filename = f'Torgnon_gpp_{year}{month}{day}.tif'
            # Get the full paths
            old_file = os.path.join(folder_path, filename)
            new_file = os.path.join(folder_path, new_filename)
            # Rename the file
            os.rename(old_file, new_file)
            print(f'Renamed: {old_file} -> {new_file}')